### Commands

In [16]:
from firecloud import api as firecloud_api
import fc_interface

In [17]:
%load_ext autoreload
%autoreload 2

In [69]:
### Parameters
namespace = "nci-mimoun-bi-org"
workspace = "CCLF_TSCA"
paths_to_samples_info = "paths_to_batches_info.xlsx"
google_bucket_id = "fc-35446f22-ea37-483a-bd6c-5e9fc56851ff"
batches_info = pd.read_excel(paths_to_samples_info)

#### Deletions

In [ ]:
### Delete unnecessary attributes in samples
attrs = ["annotate_variants_vcf", "annotate_variants_vcf_index", "oncotated_maf", "mutect2_vcf_index", "match_normal_oncotated_maf", "gnomad_vcf", "merged_vcfs", "match_primary_tumor_bam_file"]
for idx, row in all_samples.iterrows():
    fc_interface.delete_entity_attributes(namespace, workspace, "sample", row['entity:sample_id'], attrs)

In [ ]:
### Delete method repostory
firecloud_api.delete_repository_method('tsca', 'CreatePoN', 15)

In [ ]:
### Delete method repository
firecloud_api.delete_repository_method('tsca', 'Mutect2_for_Normal', 1)

In [ ]:
### Delete multiple method repositories
for i in range(10, 23):
    firecloud_api.delete_repository_method('tsca', 'CreatePoN', i)

In [ ]:
### Delete method config
firecloud_api.delete_workspace_config(namespace, workspace, "tsca", "Mutect2_for_Normal")

#### Pair creation

In [ ]:
### Create pairs
all_samples = fc_interface.get_samples_multiple_batches(paths_to_samples_info, google_bucket_id)
pairs_list = fc_interface.create_pairs_list(all_samples)
blacklist = ["DW039-Tumor-SM-DB2IF"]
clean_pairs_list = pairs_list[ ~pairs_list['case_sample_id'].isin(blacklist)]

In [ ]:
### Upload pairs
res = fc_interface.upload_pairs(namespace, workspace, clean_pairs_list)

In [ ]:
# ### Delete unnecessary attributes in pairs
# ### NOT WOKING
# all_samples = fc_interface.get_samples_multiple_batches(paths_to_samples_info, google_bucket_id)
# pairs_list = fc_interface.create_pairs_list(all_samples)
# attrs = ["__annotate_variants_vcf", "__annotate_variants_vcf_index", \
#          "__scattered_intervals", "__tmp_mutect_vcf2", "_tmp_mutect_vcf_index2", \
#          "__tmp_mutect_vcf2", "_tmp_mutect_vcf_index2"]
# for idx, row in pairs_list.iterrows():
#     fc_interface.delete_entity_attributes(namespace, workspace, "pair", row['entity:pair_id'], attrs)

In [ ]:
### Upload metadata for given batch
fc_interface.update_batch_metadata('TSCA21', \
                                  '/xchip/clf/seq_data/processed_for_fh/tsca21_201708_SN0128552/tsca21_201708_SN0128552.import_samples.txt', \
                                      paths_to_samples_info, \
                                        namespace, workspace, google_bucket_id)

In [ ]:
### Delete sample_set
fc_interface.delete_sample_set(namespace, workspace, 'TSCA1213')

In [ ]:
### All paths to batch info
# pd.read_excel('paths_to_batches_info.xlsx').path_to_samples_info.tolist()

In [ ]:
### Create cumulative PoN (all batches)
batches_info = pd.read_excel(paths_to_samples_info)
PoN = fc_interface.create_panel_of_normals(batches_info.path_to_samples_info.tolist(), -1, 'Cum_PoN_21_all')
fc_interface.upload_pon(PoN, 'CumPoN21', namespace, workspace)

In [ ]:
### Create PoN from normals of only one batch (no samples from other batches)
batches_info = pd.read_excel(paths_to_samples_info)
tsca21_path = batches_info.path_to_samples_info.tolist()[-1]
PoN = fc_interface.create_panel_of_normals([tsca21_path], -1, 'PoN_TSCA21_Normals')
fc_interface.upload_pon(PoN, 'PoN_TSCA21_Normals', namespace, workspace)

In [ ]:
### Delete pair set
fc_interface.delete_pair_set(namespace, workspace, 'TSCA1213')

In [ ]:
### Delete sample set
fc_interface.delete_sample_set(namespace, workspace, 'TSCA1213')

In [ ]:
### Download method configs
fc_interface.download_method_configs(namespace, workspace)

In [ ]:
### Upload pairsets
tn_pairsets, tp_pairsets = fc_interface.create_pairsets(all_samples, clean_pairs_list)
fc_interface.upload_pairsets(namespace, workspace, tn_pairsets, "TN")
fc_interface.upload_pairsets(namespace, workspace, tp_pairsets, "TP")

In [ ]:
### Delete all pairs
for idx, row in clean_pairs_list.iterrows():
    res = fc_interface.delete_pair(namespace, workspace, row['entity:pair_id'])
    print(res)

In [ ]:
all_samples = fc_interface.get_samples_multiple_batches(paths_to_samples_info, google_bucket_id)
pairs_list = fc_interface.create_pairs_list(all_samples)
blacklist = ["DW039-Tumor-SM-DB2IF"]
clean_pairs_list = pairs_list[ ~pairs_list['case_sample_id'].isin(blacklist)]

### Update pair attributes
update_pair_attrs(namespace, workspace, clean_pairs_list, attrs=['control_sample_tsca_id'])

In [ ]:
### Create PoN from all normals
batches_info = pd.read_excel(paths_to_samples_info)
all_paths = batches_info.path_to_samples_info.tolist()
PoN = fc_interface.create_panel_of_normals(all_paths, -1, 'Cum_PoN_21_all')
fc_interface.upload_pon(PoN, 'Cum_PoN_21_all', namespace, workspace)

In [6]:
### Create PoN with normals from batch + other normals
batches_info = pd.read_excel(paths_to_samples_info)
all_paths = batches_info.path_to_samples_info.tolist()
# PoN = fc_interface.create_panel_of_normals(all_paths, -1, 'Cum_PoN_21_all')
# fc_interface.upload_pon(PoN, 'Cum_PoN_21_all', namespace, workspace)

In [4]:
# batch_id = 'TSCA21'
# batch_path = batches_info.loc[batches_info.tsca_id==batch_id, 'path_to_samples_info'].item()
# pd.read_table(batch_path)
# batch_id, paths, N, name

In [21]:
# Retrieve samples before export to FC
tsca21 = \
    fc_interface.\
        prepare_batch_samples_for_metadata_export('/xchip/clf/seq_data/processed_for_fh/tsca21_201708_SN0128552/tsca21_201708_SN0128552.import_samples.txt', \
                            'TSCA21', google_bucket_id)

In [ ]:
### Update WDLs
fc_interface.download_remote_wdls(namespace, workspace)

---

---

In [1]:
import glob

In [19]:
walkupseq_files = glob.glob('walkupseq_files/*')

In [54]:
def prepare_cohorts_for_metadata_export(paths_to_samples_info, google_bucket_id):
    """Creates sample sets corresponding to cohorts for Firecloud export.
    Writes:
        - in cohort_files: fc_upload_sample_set_{cohort_name}.txt
    """
    # Files 
    # TODO: use glob to read all paths in a given folder
    walkupseq_files = ['walkupseq_files/tsca21_walkupseq_201708_sample_info.txt', \
                   'walkupseq_files/tsca20_walkupseq_201707_sample_info.txt', \
                   'walkupseq_files/tsca19_walkupseq_201706_sample_info.txt']
    
    # Create df with samples (walkupseq), including cohorts the samples belong to
    dfs = []
    for f in walkupseq_files:
        df_tmp = pd.read_table(f)
        dfs.append(df_tmp)
    
    walkupseq = pd.concat(dfs, axis=0).rename(columns={'Collection': 'cohort'})
    
    # All samples, without cohort data
    all_samples = fc_interface.get_samples_multiple_batches(paths_to_samples_info, google_bucket_id)
    
    # Add cohort data to all samples
    data = pd.merge(all_samples, walkupseq[['External Sample ID', 'Collection']], \
             left_on='external_id_validation', \
             right_on='External Sample ID') \
            .drop(['External Sample ID'], axis=1) \

    # Save files
    for cohort_name in data.cohorts.unique():
        cohort = cohorts[ cohorts.cohort==cohort_name ]
        # Prepare for FC export format
        cohort['membership:sample_set_id'] = cohort_name
        # Only keep columns necessary
        cohort = cohort[['membership:sample_set_id', 'enitity:sample_id']].\
                        rename(columns={'enitity:sample_id': 'sample_id'})
        cohort.to_csv('cohort_files/fc_upload_sample_set_%s.txt'%cohort_name, index=None, sep="\t")
        
    return data

In [49]:
cohorts = create_cohorts()

In [51]:
cohort_names = cohorts.cohort.unique()

In [53]:
cohort_names

array(['Cancer Cell Line Factory (CCLF) / Andy Aguirre/Kulke (DFCI) - Pancreas',
       'Cancer Cell Line Factory (CCLF) / Adam Bass/Kulke (DFCI) - Esophageal, Gastric',
       'Cancer Cell Line Factory (CCLF) / Jochen Lorch (DFCI) - Head and Neck',
       'Cancer Cell Line Factory (CCLF) / Keith Ligon (DFCI) - Neuro',
       'Cancer Cell Line Factory (CCLF) / Nir Hacohen (MGH) - Colon',
       'Cancer Cell Line Factory (CCLF) / Hong/Crompton/Katie Janeway(DFCI) - Pediatric Tumors',
       'Cancer Cell Line Factory (CCLF) / Sabina Signoretti (DFCI) - RCC',
       'Cancer Cell Line Factory (CCLF) / David Tuverson (CSHL) - Pancreas',
       'Cancer Cell Line Factory (CCLF) / Charles Yoon (DFCI) - Melanoma',
       'Cancer Cell Line Factory (CCLF) / Weinstock/Soiffer (DFCI) - TCell',
       'Cancer Cell Line Factory (CCLF) / Lisa Henske (BWH) - LAM, rare diseases',
       'Cancer Cell Line Factory (CCLF) / Hui Gao (NIBRI) - PDX Collection',
       'Cancer Cell Line Factory (CCLF) / Sid Pu

In [43]:
all_samples.columns

Index(['entity:sample_id', 'participant_id', 'clean_bam_file_capture',
       'external_id_validation', 'aggregation_product_name_validation',
       'bsp_sample_id_validation', 'stock_sample_id_validation', 'sample_type',
       'picard_aggregation_type_validation', 'processed_subtype_validation',
       'source_subtype_validation', 'squid_sample_id_validation',
       'tumor_subtype', 'short_letter_code', 'bam_filename', 'bai_filename',
       'clean_bai_file_capture', 'tsca_id'],
      dtype='object')

In [75]:
fc_interface.prepare_all_data_for_metadata_export('TSCA14', \
                                                  '/xchip/clf/seq_data/processed_for_fh/tsca14_201611_SN0108736/tsca14_201611_SN0108736.import_samples.txt', \
                                                  'paths_to_batches_info.xlsx', \
                                                  google_bucket_id)

73 Participants in this batch
95 Samples in this batch


/Users/mimoun/production_fc/metadata_exports/fc_interface.py:417: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  cohort['membership:sample_set_id'] = cohort_name


KeyError: "['enitity:sample_id'] not in index"